In [ ]:
# !pip install elasticsearch
!pip install pororo

In [54]:
import os
import sys
import json

import elasticsearch
from elasticsearch import Elasticsearch
import pandas as pd
import pororo
import torch

In [14]:
# 엘라스틱서치 오브젝트 만들기
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [15]:
# connection check
es.ping()

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


True

In [10]:
# 인덱스 만들기
es.indices.create(index='my-foo', ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'my-foo'}

In [16]:
# 인덱스 보기
res = es.indices.get_alias("*")
for Name in res:
  print(Name)

.apm-agent-configuration
.kibana_7.15.1_001
.kibana-event-log-7.15.1-000001
.kibana_task_manager_7.15.1_001
my-foo
.apm-custom-link
wiki_documents
.tasks


/tmp/ipykernel_1498/1551562474.py:2: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.indices.get_alias("*")
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: this request accesses system indices: [.apm-agent-configuration, .kibana_7.15.1_001, .kibana_task_manager_7.15.1_001, .apm-custom-link, .tasks], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchWarning)


In [79]:
# 인덱스 삭제
es.indices.delete(index='wiki_documents', ignore=[400, 404])

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True}

# Prepare data for Elastic search

In [9]:
with open('../data/wikipedia_documents.json') as f:
  data = json.load(f)

In [10]:
count = 0
for k,v in data.items():
  print(k, v)
  count += 1
  if count == 1:
    break

0 {'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.', 'corpus_source': '위키피디아', 'url': 'TODO', 'domain': None, 'title': '나라 목록', 'author': None, 'html': None, 'document_id': 0}


In [11]:
len(data)

60613

In [14]:
df = pd.read_json('../data/wikipedia_documents.json').T

In [15]:
df.head()

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,None,아오조라 문고,None,None,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,None,아오조라 문고,None,None,4


In [16]:
df.nunique()

text             56737
corpus_source        1
url                  1
domain               0
title            31755
author               0
html                 0
document_id      60613
dtype: int64

### 중복 text 레코드 제거

In [17]:
df = df.drop_duplicates(subset=['text'])
df.nunique()

text             56737
corpus_source        1
url                  1
domain               0
title            31726
author               0
html                 0
document_id      56737
dtype: int64

In [80]:
len(df)

56737

In [64]:
print(df.loc[df['document_id']==23174]['text'])

23174    순항 유도탄에 적용되는 추진기관은 일반적으로 생각하는 로켓이 사용되지 않는다. 이는...
Name: text, dtype: object


In [19]:
df2 = df.to_dict('records')

In [70]:
df2[56500]

{'text': '1945년 11월 30일 서방 연합군에서는 서베를린 진입용 폭 32 km 항로를 소련군과 합의했다  베를린 진입 가능 항로는 함부르크 항로(북서부), 뷔케베부크 항로(서부, 하노버 방면), 프랑크푸르트 항로(남서부)였다. 1945년 12월 31일 협의에 따라서  사용 조건이 확정되었다. 해당 항로 3곳은 자유 항행 가능, 시간 제한 없음, 서방 연합군 국적기의 경우 이용 통보 없음이 보장되었다. \n\n영국군이 1948년 4월에 소규모 공수작전을 실행해 보았고, 1948년 6월에 크니커 작전(Operation Knicker)이라는 이름으로 공수작전을 해 본 경험이 도움이 되었다 베를린 주둔 영국 공군 총사령관 레지널드 와이트(Reginald Waite)는 공수작전 중에 서베를린 민간인에게도 하계 물자를 추가로 공급할 수 있을 것이라고 판단했고, 1948년 6월 24일 루시우스 클레이에게 통보했다. 다음 날 베를린의 선출직 시장인 에른스트 로이터를 소환해서 베를린 시민들이 봉쇄 중 공수로 전달할 수 있는 제한된 물자로 생활할 수 있는지 질문했다. 에른스트 로이터는 빌리 브란트와 함께 가서 공수 작전이 시행되면 자신은 베를린시를 책임질 수 있으며, 베를린의 자유를 위해서 필요한 희생을 받아들일 수 있다고 밝혔다. 에른스트 로이터 시장은 루시우스 클레이 장군의 결정을 존경하지만, 공수작전의 성공 가능성을 높게 보지는 않는다고 밝혔다.   클레이 장군은 유럽 주둔 미 공군 사령관 커티스 르메이에게 공수작전 시작을 통보했다.\n\n6월 26일 미국 공군의 첫 수송기가 프랑크푸르트(라인마인 공군기지) 및 비스바덴에서 베를린 템펠호프 공항으로 날아오면서 공수작전이 시작되었다(Operation Vittles 또는 Operation Proviant). 영국 공군은 6월 28일 분스토르프 비행장에서 가토 비행장으로 더글러스 C-47 수송기를 출발하면서 역시 공수작전에 참가했다(Operation Plainfare, 이후 Operation Carter Patter

In [69]:
df_rest = df2[56500:]

In [23]:
def generator(df2):
  for c, line in enumerate(df2):
    yield {
      '_index': 'wiki_documents',
      '_type': '_doc',
      '_id': line.get('document_id', None),
      '_source': {
        'title': line.get('title', ''),
        'text': line.get('text', '')
      }
    }
  raise StopIteration

In [24]:
wiki_generator = generator(df2)

In [25]:
wiki_generator

<generator object generator at 0x7f24b95f80b0>

# Settings and Mappings

In [26]:
# Settings = {
#   'settings':{
#     'number_of_shards':1,
#     'number_of_replicas':1,
#   },
#   'mappings':{
#     'properties':{
#       'title':{
#         'type':'text'
#       },
#       'text':{
#         'type':'text'
#       }
#     }
#   }
# }

CONFIG = {
  'settings':{
    'number_of_shards':1,
    'number_of_replicas':1,
    'analysis':{
      'analyzer': {
        'nori_analyzer':{
          'type': 'custom',
          'tokenizer': 'nori_tokenizer'
        }
      },
    }
  },
  'mappings':{
    'properties':{
      'title':{
        'type':'text'
      },
      'text':{
        'type':'text',
        "analyzer": "nori_analyzer"
      }
    }
  }
}

# CONFIG = {
#   'settings':{
#     'number_of_shards':1,
#     'number_of_replicas':1,
#     'analysis':{
#       'analyzer': {
#         'nori_analyzer':{
#           'type': 'custom',
#           'tokenizer': 'nori_tokenizer',
#           'filter':[
#             'nori_part_of_speech_filter'
#           ]
#         }
#       },
#       'filter':{
#         'nori_part_of_speech_filter': {
#           'type': 'nori_part_of_speech',
#         }
#       }
#     }
#   },
#   'mappings':{
#     'properties':{
#       'title':{
#         'type':'text'
#       },
#       'text':{
#         'type':'text',
#         "analyzer": "nori_analyzer",
#         'search_analyzer':'nori_analyzer'
#       }
#     }
#   }
# }


In [81]:
es.indices.create(index='wiki_documents', ignore=[400,404], body=CONFIG)

/tmp/ipykernel_10274/2280445778.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index='wiki_documents', ignore=[400,404], body=CONFIG)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki_documents'}

# Upload the Data into Elastic Search

In [82]:
from elasticsearch import helpers
gen = generator(df2)
gen_rest = generator(df_rest)

In [83]:
try:
  res = helpers.bulk(es, gen, chunk_size = 1)
  # print(res)
except Exception as e:
  print(e)
  print('Done')

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


generator raised StopIteration
Done


In [75]:
try:
  helpers.bulk(es, gen_rest, chunk_size = 1)
except Exception as e:
  print(e)
  print('Done')

generator raised StopIteration
Done


In [50]:
count = 0
while True:
  try:
    next(gen)
    count += 1
  except Exception:
    print(Exception)
    break
print(count)

<class 'Exception'>
56735


In [19]:
res

{'.kibana_task_manager_7.15.1_001': {'aliases': {'.kibana_task_manager': {},
   '.kibana_task_manager_7.15.1': {}}},
 '.kibana-event-log-7.15.1-000001': {'aliases': {'.kibana-event-log-7.15.1': {'is_write_index': True}}},
 'wiki_documents': {'aliases': {}},
 '.apm-agent-configuration': {'aliases': {}},
 '.apm-custom-link': {'aliases': {}},
 '.kibana_7.15.1_001': {'aliases': {'.kibana': {}, '.kibana_7.15.1': {}}},
 '.tasks': {'aliases': {}}}

# queries

In [84]:
es.count(index = 'wiki_documents')


{'count': 56736,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [87]:
es.get(index='wiki_documents', id=56737)

{'_index': 'wiki_documents',
 '_type': '_doc',
 '_id': '56737',
 '_version': 1,
 '_seq_no': 52910,
 '_primary_term': 1,
 'found': True,
 '_source': {'title': '왼손잡이 에렌',
  'text': '; 아사쿠라 코이치(朝倉光一)\n: 이 이야기의 주인공이다. 만들기는 좋아하지만 뛰어난 재능이 없는 것을 자각하고 광고 대리점의 디자이너가 되어서 등신대의 성공을 하고 싶다고 생각하고 있는 청년이다. \n; 야마기시 에렌(山岸エレン)\n: 그림의 재능을 가지면서 아버지의 죽음을 계기로 그리는 것을 그만둔 소녀이면 왼손잡이이다. \n; 카토 사유리(加藤さゆり)\n: 무엇이든 평균 이상은 능숙하게 다르며, 만만찮은 인생 설계야 말로 행복의 길이라고 믿고 있다. \n; 키시 아야노(岸あやの)\n: 대기업 ‘AK’의 사장의 딸이자 키시 아카리의 언니이다. 왼손잡이이다.\n; 키시 아카리(岸あかり)\n: 엄청 인기 있는 모델이다. 키시 아야노의 여동생이다. 왼손잡이이다.'}}

In [21]:
es.get(index='wiki_documents', id=0)

{'_index': 'wiki_documents',
 '_type': '_doc',
 '_id': '0',
 '_version': 1,
 '_seq_no': 0,
 '_primary_term': 1,
 'found': True,
 '_source': {'title': '나라 목록',
  'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'}}

In [46]:
es.search(index='wiki_documents', body={'size':1, 'query':{'match':{'text':'국제적인 승인을 널리'}}})

/tmp/ipykernel_6464/706394350.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(index='wiki_documents', body={'size':1, 'query':{'match':{'text':'국제적인 승인을 널리'}}})
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'took': 35,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1775, 'relation': 'eq'},
  'max_score': 23.013771,
  'hits': [{'_index': 'wiki_documents',
    '_type': '_doc',
    '_id': '0',
    '_score': 23.013771,
    '_source': {'title': '나라 목록',
     'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'}}]}}

In [17]:
es.indices.get_mapping(index='wiki_documents')

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'wiki_documents': {'mappings': {'properties': {'text': {'type': 'text'},
    'title': {'type': 'text'}}}}}

In [20]:
settings = {
  'settings':{
    'number_of_shards':1,
    'number_of_replicas':1,
    'analysis':{
      'analyzer': {
        'nori_analyzer':{
          'type': 'custom',
          'tokenizer': 'nori'
        }
      }
    }
  }
}

In [ ]:
es.indices.put_settings(body=settings, index='wiki_documents')

In [18]:
mapping = {
'mappings':{
    'properties':{
      'title':{
        'type':'text'
      },
      'text':{
        'type':'text',
        "analyzer": "nori_analyzer"
      }
    }
  }
}

In [ ]:
es.indices.put_mapping(body=mapping, index='wiki_documents')

In [16]:
es.indices.analyze(
  body={
    'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.',
    'analyzer':'nori',
  }
)

{'tokens': [{'token': '문서',
   'start_offset': 2,
   'end_offset': 4,
   'type': 'word',
   'position': 1},
  {'token': '나라',
   'start_offset': 6,
   'end_offset': 8,
   'type': 'word',
   'position': 3},
  {'token': '목록',
   'start_offset': 9,
   'end_offset': 11,
   'type': 'word',
   'position': 4},
  {'token': '이',
   'start_offset': 11,
   'end_offset': 12,
   'type': 'word',
   'position': 5},
  {'token': '세계',
   'start_offset': 17,
   'end_offset': 19,
   'type': 'word',
   'position': 8},
  {'token': '206',
   'start_offset': 20,
   'end_offset': 23,
   'type': 'word',
   'position': 9},
  {'token': '개',
   'start_offset': 23,
   'end_offset': 24,
   'type': 'word',
   'position': 10},
  {'token': '나라',
   'start_offset': 25,
   'end_offset': 27,
   'type': 'word',
   'position': 11},
  {'token': '현황',
   'start_offset': 31,
   'end_offset': 33,
   'type': 'word',
   'position': 14},
  {'token': '주권',
   'start_offset': 35,
   'end_offset': 37,
   'type': 'word',
   'position

In [88]:
es.search(
  index='wiki_documents', 
  body={
    'query':{
      'match':{
        'text':'용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?',
        # 'tokenizer': 'nori'
      }
    }
  },
  size=3
)

/tmp/ipykernel_11168/1677013031.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(


{'took': 19,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 21.447016,
  'hits': [{'_index': 'wiki_documents',
    '_type': '_doc',
    '_id': '20081',
    '_score': 21.447016,
    '_source': {'title': '크리스 니코스키',
     'text': 'SK에서 방출된 이후 그는 일본을 여행하며 잠시 신변을 정리하고 있었다. 그 때 좌완 선발 투수가 필요하여 니코스키에게 관심을 두고 있었던 두산 베어스가 SK 와이번스에 계약 양도를 신청하여 곧바로 맷 왓슨의 대체 용병으로 두산 베어스로 이적했고 두산 베어스의 선발진에 합류하여 비로소 첫 승을 따 냈으며  승리를 대부분 한화 이글스전에서 기록하여 한화 이글스에는 강력한 천적으로 자리매김했다.  그러나 예전부터 주로 원포인트 및 중간계투로 뛰다가 선발로 전환한 탓에 선발로는 아직 익숙하지 않아 3회까지 투구수가 70개 가까이 되는 경기가 대부분이라는 단점이 있었다.  2009년 9월 13일 KIA 타이거즈전에서 비로소 초반에 급격하게 늘어나는 투구수를 극복하여 승리를 따 내기도 했다.  두산 베어스 합류 후 12경기에 더 나서고 2009년 정규 시즌을 4승 8패, 평균 자책 3.78로 마무리했다. 완전한 선발 전업 후 후반기에 조금씩 나아지는 모습을 보였으나, 잠실야구장에서 롯데 자이언츠와 맞붙었던 2009년 9월 29일 준 플레이오프 1차전에서 무실점으로 3이닝을 호투하던 중 갑작스러운 어깨 통증으로 조기 강판당했고, 그 날 두산 베어스는 롯데 자이언츠에게 1차전을 내주었다. 진단 결과 극상근 손상 판정을 받아 포스트 시즌 전력에서 완전히 이탈하고 말았다. 

In [17]:
res = es.search(
  index='wiki_documents', 
  body={
    'query':{
      'match':{
        'text':'용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?',
        # 'tokenizer': 'nori'
      }
    }
  },
  size=3
)

/tmp/ipykernel_1498/1528268646.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(


In [18]:
res['hits']['hits'][0]['_source']['text']

'SK에서 방출된 이후 그는 일본을 여행하며 잠시 신변을 정리하고 있었다. 그 때 좌완 선발 투수가 필요하여 니코스키에게 관심을 두고 있었던 두산 베어스가 SK 와이번스에 계약 양도를 신청하여 곧바로 맷 왓슨의 대체 용병으로 두산 베어스로 이적했고 두산 베어스의 선발진에 합류하여 비로소 첫 승을 따 냈으며  승리를 대부분 한화 이글스전에서 기록하여 한화 이글스에는 강력한 천적으로 자리매김했다.  그러나 예전부터 주로 원포인트 및 중간계투로 뛰다가 선발로 전환한 탓에 선발로는 아직 익숙하지 않아 3회까지 투구수가 70개 가까이 되는 경기가 대부분이라는 단점이 있었다.  2009년 9월 13일 KIA 타이거즈전에서 비로소 초반에 급격하게 늘어나는 투구수를 극복하여 승리를 따 내기도 했다.  두산 베어스 합류 후 12경기에 더 나서고 2009년 정규 시즌을 4승 8패, 평균 자책 3.78로 마무리했다. 완전한 선발 전업 후 후반기에 조금씩 나아지는 모습을 보였으나, 잠실야구장에서 롯데 자이언츠와 맞붙었던 2009년 9월 29일 준 플레이오프 1차전에서 무실점으로 3이닝을 호투하던 중 갑작스러운 어깨 통증으로 조기 강판당했고, 그 날 두산 베어스는 롯데 자이언츠에게 1차전을 내주었다. 진단 결과 극상근 손상 판정을 받아 포스트 시즌 전력에서 완전히 이탈하고 말았다. 플레이오프가 끝난 이후 이닝 소화 능력이 떨어진다는 이유로 두산 베어스가 재계약을 포기하면서 대한민국을 떠나게 되었다.  두산에서의 마지막이 좋지 않았지만, 그가 두산에 입단하여 2군에서 불펜 피칭을 했을 때 그를 지켜보았던 박종훈 당시 2군 감독은 니코스키를 두고 성격이 좋았다고 하였다.'

In [17]:
from datasets import Dataset, DatasetDict

df = pd.DataFrame({'질문':['질문1', '질문2', '질문3'], '답변':['답변1', '답변2', '답변3']})
DatasetDict({"validation": Dataset.from_pandas(df)})

DatasetDict({
    validation: Dataset({
        features: ['질문', '답변'],
        num_rows: 3
    })
})

# Entity Enrichment

In [55]:
ner = pororo.Pororo(task='ner', lang='ko').to('cuda')
nered_question = ner('대한민국의 대통령은 누구인가?')
nered_question

AttributeError: 'PororoBertCharNer' object has no attribute 'to'

In [44]:
match_phrases = []
for word in nered_question:
  if word[1] != 'O':
    match_phrases.append({'match_phrase':{'text': word[0]}})
print(*match_phrases)

{'match_phrase': {'text': '대한민국'}} {'match_phrase': {'text': '대통령'}}


In [51]:
res = es.search(
  index='wiki_documents', 
  body={
    'query':{
      'bool':{
        'should':[
          {
            'match':{
              'text':'대한민국의 대통령은 누구인가?',
            }
          },
          # {'match_phrase': {'text':'용병'}},
          *match_phrases
        ]
      }
    }

  },
  size=3
)

/tmp/ipykernel_1498/1622811465.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(


In [52]:
res

{'took': 12,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 33.414112,
  'hits': [{'_index': 'wiki_documents',
    '_type': '_doc',
    '_id': '42655',
    '_score': 33.414112,
    '_source': {'title': '가토 다쓰야 (저널리스트)',
     'text': "2016년 1월 29일자로 일본 산케이 신문 1면기사로 책발간 소식을 알리고 《왜 나는 한국에 승리하였나. 박근혜 정권과의 500일 전쟁》이라는 제목의 책을 발간개시하였다.  책에서 그는 대한민국은 중세 국가와 마찬가지라 하고 대한민국은 대통령과 국민 감정에 따라 자의적으로 법이 해석된다고 했다. 그는 이어서 대한민국 검찰은 박근혜의 동태와 여론의 반응을 살피기 바빴고, 그래서 '과연 대한민국은 민주주의 가치를 공유하며 법치주의를 실천하고 있다고 말할 수 있는가’라며 대한민국을 비난했으며, '오히려 일본을 악의 제국으로 매도하며 대한민국이 일방적인 견해를 시사하고 있는 것 아닌가”라고 했다.  가토 다쓰야는 이 책으로 2016년 야마모토 시치헤이 상(山本七平賞)을 수상하였다."}},
   {'_index': 'wiki_documents',
    '_type': '_doc',
    '_id': '53821',
    '_score': 32.705547,
    '_source': {'title': '대한민국 제19대 대통령 선거 민중연합당 후보 선출',
     'text': '선거명=|국가=대한민국|유형=대통령|이전선거=대한민국 제18대 대통령 선거 통합진보당 후보 경선|이전선거_연도=2012년|선거일=2017년 3월 21일~25일|차기선거=대

In [71]:
from nltk import ne_chunk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [74]:
pos_tag(word_tokenize('1981년 7월 대한민국의 전두환 대통령이 처음으로 인도네시아를 방문하였으며, 이듬해 1982년 10월에는 인도네시아 수하르토 대통령이 방한하였다. 1988년 11월에는 노태우 대통령이 인도네시아를 방문하였고, 이어 1994년 11월 김영삼 대통령도 인도네시아를 찾았다'))

[('1981년', 'CD'),
 ('7월', 'CD'),
 ('대한민국의', 'NN'),
 ('전두환', 'NNP'),
 ('대통령이', 'NNP'),
 ('처음으로', 'NNP'),
 ('인도네시아를', 'NNP'),
 ('방문하였으며', 'NNP'),
 (',', ','),
 ('이듬해', 'VBD'),
 ('1982년', 'CD'),
 ('10월에는', 'CD'),
 ('인도네시아', 'NN'),
 ('수하르토', 'NNP'),
 ('대통령이', 'NNP'),
 ('방한하였다', 'NNP'),
 ('.', '.'),
 ('1988년', 'CD'),
 ('11월에는', 'CD'),
 ('노태우', 'NN'),
 ('대통령이', 'NNP'),
 ('인도네시아를', 'NNP'),
 ('방문하였고', 'NNP'),
 (',', ','),
 ('이어', 'VBD'),
 ('1994년', 'CD'),
 ('11월', 'CD'),
 ('김영삼', 'NN'),
 ('대통령도', 'NNP'),
 ('인도네시아를', 'NNP'),
 ('찾았다', 'NN')]

In [68]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /opt/ml/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True